https://radimrehurek.com/gensim/models/keyedvectors.html

In [1]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from pymystem3 import Mystem
from multiprocessing import Pool
import time

In [2]:
word_vectors = KeyedVectors.load_word2vec_format('/data/gensim/news_0_300_2.bin.gz', binary=True)
word_vectors.init_sims(replace=True)

In [3]:
mystem = Mystem()

In [4]:
mystem_part = {
    'A': 'ADJ', # прилагательное
    'ADV': 'ADV', # наречие
    'ADVPRO': 'ADV', # местоименное наречие
    'ANUM': 'ADJ', # числительное-прилагательное
    'APRO': 'DET', # местоимение-прилагательное
    'COM': 'ADJ', # часть композита - сложного слова
    'CONJ': 'SCONJ', # союз
    'INTJ': 'INTJ',	# междометие
    'NUM': 'NUM', # числительное
    'PART': 'PART',	# частица
    'PR': 'ADP', # предлог
    'S': 'NOUN', # существительное
    'SPRO': 'PRON',
    'V': 'VERB', # глагол
}

In [5]:
def lemm(word):
    a = mystem.analyze(word)
    
    if len(a) == 0:
        return word
    
    analysis = a[0]['analysis']
    
    if len(analysis) == 0:
        return word
    
    gr = analysis[0]
    parts = gr['gr'].split("=")
    
    if len(parts) == 0:
        return word
    
    parts2 = parts[0].split(",")
        
    if len(parts2) == 0:
        return word
    
    part = parts2[0]
        
    if part in mystem_part:
        return "%s_%s" % (analysis[0]['lex'], mystem_part[part])
    else:
        return word

In [36]:
def similar(word, lemming=True):
    try:
        if lemming:
            w = lemm(word)
        else:
            w = word
        sim = word_vectors.most_similar(positive=[w])
        return list(map(lambda x: x[0], sim))
    except Exception as e:
        return [word]

In [53]:
def extractSemanticGroup(seq):
    token_seq = map(lambda x: map(lambda y: lemm(y), x.split(" ")), seq)
    res_seq = []
    sem_groups = {}
    for words in token_seq:
        res_tokens = []
        for token in words:
            if token not in sem_groups:
                sims = similar(token, False)
                sem_groups[token] = token
                res_tokens.append(token)
                for sim in sims:
                   sem_groups[sim] = token
            else:
                res_tokens.append(sem_groups[token])
        res_seq.append(res_tokens)
    result = map(lambda x: " ".join(map(lambda y: y.split("_")[0], x)), res_seq)
    return list(result)

In [54]:
def unic_count(seq):
    total = []
    for s in seq:
        total.extend(s.split(" "))
    return len(set(total))

In [55]:
news = [
    "свидетель редкий природный явление становиться житель москва первый день февраль рано утро многий разбудить раскат гром настоящий зимний гроза сопровождаться молния порыв ветер метр секунда ледяной дождь новый месяц",
    "обвинять государственный измена домохозяйка вязьма светлана давыдов освобождать подписка невыезд страница фейсбук сообщать адвокат женщина иван павлов ходатайство защита изменение мера пресечение удовлетворять следователь взять подписка невыезд мать смочь вернуться ребенок сообщать защитник давыдова судя адвокат новость",
    "пока ребенок школа выращивать фасоль выращивать грудь говорить героиня популярный сериал глупо звучать грудь выращивать подращивать немного итак нужно делать качать мышца грудь поддерживать грудной мышца любой мышца тело подкачивать укреплять интернет смочь узнавать упражнение тренажер дом коврик помогать прорабатывать мышца грудь мышца окрепнуть подрастать грудь выглядеть пить молоко бабушка советовать капуста грудь расти"
]

In [56]:
start = time.time()
res = extractSemanticGroup(news)
print(res)
print("time: %s s" % (time.time() - start))

['свидетель редкий природный явление становиться житель москва первый день февраль рано утро многий разбудить раскат гром настоящий зимний гроза сопровождаться молния порыв порыв метр секунда ледяной гроза новый день', 'обвинять государственный измена домохозяйка вязьма светлана давыдов освобождать подписка подписка страница страница сообщать адвокат женщина иван павлов ходатайство защита изменение мера пресечение удовлетворять следователь взять подписка подписка мать смочь вернуться ребенок сообщать защитник давыдов судя адвокат новость', 'пока ребенок школа выращивать фасоль выращивать грудь говорить героиня популярный сериал глупо звучать грудь выращивать подращивать немного итак нужно делать качать мышца грудь поддерживать грудной мышца любой мышца тело подкачивать укреплять интернет нужно узнавать упражнение тренажер дом коврик помогать прорабатывать мышца грудь мышца окрепнуть подрастать грудь выглядеть пить молоко мать советовать капуста грудь расти']
time: 1.4393730163574219 s


In [57]:
unic_count(res)

96

In [58]:
unic_count(news)

103

In [59]:
import json
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [60]:
news = []
with open('/data/kasandra/year/10k.test.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [61]:
content = list(map(lambda x: x.content, news))

In [62]:
unic_count(content)

66845

In [64]:
start = time.time()
sem_content = extractSemanticGroup(content)
print("time: %s s" % (time.time() - start))

time: 696.5620369911194 s


In [65]:
unic_count(sem_content)

36093

In [47]:
start = time.time()
sem_content = extractSemanticGroup(content, 5)
print("time: %s s" % (time.time() - start))

time: 2005.195344209671 s


In [48]:
unic_count(sem_content)

36093